<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação das bibliotecas

In [ ]:
!pip install -qU langchain-pinecone pinecone-notebooks requests langchain-groq langchain_huggingface langchain_mistralai

## Configuração de Variaveis

In [ ]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')
  MISTRAL_AI_KEY    = userdata.get('MISTRAL_AI_KEY')
  MISTRAL_AI_KEY2   = userdata.get('MISTRAL_AI_KEY2')

settings = Settings()

### Geração de embeddings e indíces do banco vetorial

In [ ]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [ ]:
class FineTunedEmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'winderfeld/cc-uffs-ppc-ft-test-multiqa'
    self.dimension = 768



finetune_embeddings_model = FineTunedEmbeddingModelSpecs()

In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [ ]:
import time

index_name = "index-ppc-markdown-ids-metadata-in-content-v4"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=finetune_embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
index_name = "index-ppc-markdown-dev"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index_naive = pc.Index(index_name)

In [ ]:
index_name = "index-ppc-markdown-dev-finetuned-naive"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=finetune_embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index_naive_ft = pc.Index(index_name)

## Vector store local

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

In [ ]:
ft_embeddings = HuggingFaceEmbeddings(model_name=finetune_embeddings_model.name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store_advanced = PineconeVectorStore(index=index, embedding=ft_embeddings)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store_naive = PineconeVectorStore(index=index_naive, embedding=embeddings)

In [ ]:
vector_store_naive_ft = PineconeVectorStore(index = index_naive_ft, embedding = ft_embeddings)

## Geração da resposta

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_mistralai import ChatMistralAI


# llm = ChatGroq(
#     model="mixtral-8x7b-32768",
#     temperature=0,
#     groq_api_key=settings.GROQ_API_KEY,
#     max_retries=4
# )

llm = ChatMistralAI(model='open-mixtral-8x22b', #'mistral-large-2407',
                    temperature=0,
                    max_retries=4,
                    api_key=settings.MISTRAL_AI_KEY2)


CHUNK_SPLITTER = '\n<|FIM_DO_CHUNK|>\n'



def query_transformation(query):
   query_transformation_llm = ChatGroq(model = 'mixtral-8x7b-32768', #model='open-mixtral-8x7b', #'mistral-large-2407',
                      temperature=0,
                      max_retries=4,
                      api_key=settings.GROQ_API_KEY)

   system_prompt = """Você é um transformador de consultas acadêmicas. Sua única função é reformatar perguntas em strings padronizadas.

TAREFA:
Transforme a pergunta do usuário em uma string padronizada. Jamais responda à consulta ou adicione qualquer conteúdo extra.

REGRAS:
1. Para perguntas sobre dados tabulares (ementas, carga horária, fichas de avaliação, atividades, componentes curriculares):
  → Use estrutura "DISCIPLINA | CATEGORIA"

2. Para outras perguntas não tabulares:
  → Reescreva a mesma pergunta usando sinônimos e estrutura gramatical diferente, mantendo exatamente o mesmo significado
  → A nova versão deve ser mais formal e concisa, mas perguntando a mesma coisa
  → Exemplo: "Como eu faço para me inscrever nas Atividades Complementares?" → "Qual o procedimento para matrícula em Atividades Curriculares Complementares"
  → Exemplo: "Gostaria de saber quando devo fazer o TCC" → "Em qual período realizar o Trabalho de Conclusão de Curso"
  → Exemplo: "Me diz como funciona o PPC" → "Qual a metodologia do Plano Pedagógico do Curso"
  → Exemplo: "Quero saber mais sobre as normas de ACC" → "Quais as diretrizes das Atividades Curriculares Complementares"

3. Sempre expanda estas siglas:
  - PPC → Plano Pedagógico do Curso
  - ACC → Atividades Curriculares Complementares
  - ORG → Organização de Computadores
  - BD → Banco de Dados
  - ENG → Engenharia
  - PGP → Planejamento e Gestão de Projetos
  - TCC → Trabalho de Conclusão de Curso

4. Mantenha termos já escritos por extenso

FORMATO DO OUTPUT:
- Retorne EXCLUSIVAMENTE a string transformada
- Não inclua aspas, explicações ou qualquer outro texto
- Não responda à consulta

EXEMPLOS:
Perguntas sobre dados tabulares:
Entrada: "Qual a ementa de ORG?"
Output: ORGANIZAÇÃO DE COMPUTADORES | EMENTA

Entrada: "Quantas horas tem BD?"
Output: BANCO DE DADOS | HORAS

Entrada: "Quais são os pré-requisitos de Teoria dos Grafos?"
Output: TEORIA DOS GRAFOS | PRÉ-REQUISITOS

Perguntas não tabulares:
Entrada: "Como funciona o PPC?"
Output: Como funciona o Plano Pedagógico do Curso


Entrada: "Qual o prazo para entrega do TCC?"
Output: Prazo de entrega do Trabalho de Conclusão de Curso"""

   human_prompt = f"Transforme esta consulta: {query}"

   prompt_template = ChatPromptTemplate.from_messages([
       ("system", system_prompt),
       ("human", human_prompt)
   ])

   chain = prompt_template | query_transformation_llm
   response = chain.invoke({})

   return response.content


def format_chunk(chunk):
  try:
    return chunk.split('</metadata>')[1] + CHUNK_SPLITTER
  except:
    return chunk + CHUNK_SPLITTER



def get_context(query, source_name, vector_store_db, ids=[]):


    results = vector_store_db.search(
        query,
        "similarity",
        k=5,
        filter={"source": source_name},
    )




    page_content = ''.join(
        [format_chunk(result.page_content) for result in results if result.metadata.get('id') not in ids]
    )

    ids = [result.metadata.get('id') for result in results if result.metadata.get('id') not in ids]

    return page_content, ids




def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua função é fornecer respostas precisas e confiáveis sobre qualquer questão relacionada ao PPC, utilizando apenas as informações contidas no documento como referência. "
                "Se a resposta para pergunta não estiver nos trechos fornecidos, simplesmente informe que a resposta não foi encontrada."
                "\n\n"
                "Responda exclusivamente em português brasileiro e formate a resposta em markdown simples."
            ),
            (
                "human",
                f"Dado os seguintes trechos do PPC: {contexto}, responda a pergunta: {pergunta}.\n\n.Lembre-se de utilizar apenas o PPC como fonte para responder. Respire fundo e valide a resposta. "
            ),
        ]
    )
    return prompt



def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    print(response)
    return response


def naive_pipeline(query, source_name):
    context, _= get_context(query, source_name, vector_store_naive)

    llm_context = context.replace(CHUNK_SPLITTER, '\n\n--------------\n\n')

    prompt = build_prompt(llm_context, query)
    response = get_completion(prompt)

    return context, response.content, "", response.response_metadata.get('token_usage')

def naive_ft_pipeline(query, source_name):
    context, _= get_context(query, source_name, vector_store_naive_ft)
    llm_context = context.replace(CHUNK_SPLITTER, '\n\n--------------\n\n')

    print(vector_store_naive_ft.__dict__)


    prompt = build_prompt(llm_context, query)
    response = get_completion(prompt)


    return context, response.content, "", response.response_metadata.get('token_usage')



def advanced_pipeline(query, source_name):
    expanded_query = query_transformation(query)
    print('Query expandida: ', expanded_query)
    query_context, ids = get_context(query, source_name, vector_store_advanced)
    print(len(ids))
    expanded_query_context, new_ids = get_context(expanded_query, source_name, vector_store_advanced, ids)
    print(len(new_ids))
    context = query_context + '\n\n' + expanded_query_context

    llm_context = context.replace(CHUNK_SPLITTER, '\n\n--------------\n\n')

    prompt = build_prompt(llm_context, query)
    response = get_completion(prompt)

    return context, response.content, expanded_query, response.response_metadata.get('token_usage')


In [ ]:
from typing import List, Dict, Tuple
import time
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_mistralai import ChatMistralAI
from langchain_core.output_parsers import CommaSeparatedListOutputParser


# Configuração dos modelos LLM
query_expansion_llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=2
)

response_llm = ChatMistralAI(
    model='open-mixtral-8x22b',
    temperature=0,
    max_retries=4,
    api_key=settings.MISTRAL_AI_KEY
)

final_response_llm = ChatMistralAI(
    model='mistral-large-latest',
    temperature=0,
    max_retries=4,
    api_key=settings.MISTRAL_AI_KEY
)

from typing import List, Dict, Tuple
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
import time

def aggregate_token_usage(usages: List[Dict]) -> Dict[str, int]:
    """
    Aggregate multiple token usage dictionaries.

    Args:
        usages: List of token usage dictionaries from LangChain responses

    Returns:
        Combined token usage dictionary
    """
    total = {
        'prompt_tokens': 0,
        'completion_tokens': 0,
        'total_tokens': 0
    }

    for usage in usages:
        if not usage:
            continue
        total['prompt_tokens'] += usage.get('prompt_tokens', 0)
        total['completion_tokens'] += usage.get('completion_tokens', 0)
        total['total_tokens'] += usage.get('total_tokens', 0)

    return total

def get_comparative_answer(query: str, pipeline_type: str = "naive") -> Tuple[str, str, str, Dict[str, int]]:
    """
    Generate comparative analysis between PPCs with detailed token usage tracking.

    Args:
        query: Original user question
        pipeline_type: Type of pipeline to use ("naive" or "advanced")

    Returns:
        - final_response: Combined analysis comparing both PPCs
        - expanded_queries: Original query plus transformed sub-queries
        - last_context: Context from last processed query
        - token_usage: Detailed token usage statistics
    """
    # Track all token usages for final aggregation
    all_token_usages = []

    # Get expanded queries and add original query
    sub_queries = get_expanded_queries(query)

    all_queries = [query] + sub_queries['queries']

    print(all_queries)

    # Initialize response tracking
    responses = {'2018': [], '2024': []}
    ppc_token_usage = {
        'ppc_2018': 0,
        'ppc_2024': 0,
    }
    last_context = ""

    # Process each query for both PPCs
    for sub_query in all_queries:
        responses, ppc_usage, last_context, query_usages = process_query_for_ppcs(
            sub_query,
            responses,
            pipeline_type
        )
        # Update PPC-specific totals
        ppc_token_usage['ppc_2018'] += ppc_usage['ppc_2018']
        ppc_token_usage['ppc_2024'] += ppc_usage['ppc_2024']
        all_token_usages.extend(query_usages)

    # Generate final comparative response
    final_response, context, final_usage = generate_final_comparative_response(query, responses)
    ppc_token_usage['final_response'] = final_usage.get('total_tokens', 0)
    all_token_usages.append(final_usage)

    # Aggregate all token usages
    total_usage = aggregate_token_usage(all_token_usages)

    # Combine totals with PPC-specific usage
    token_usage = {
        **total_usage,
        **ppc_token_usage
    }

    # Join all queries for return
    expanded_queries_str = ", ".join(all_queries)

    return final_response, expanded_queries_str, context, token_usage

def get_expanded_queries(query: str) -> Dict:
    """
    Transform original query into relevant sub-queries.

    Returns dictionary with queries and token usage.
    """
    prompt = f"""
      Instruções para Análise e Transformação de Perguntas:

      Contextualização: Tenho dois documentos PPCs e preciso fazer análises comparativas. Para isso, preciso buscar transformar as consultas do usuário em subconsultas específicas para cada documento em análise.
      Sua tarefa é receber a pergunta original e siga as etapas abaixo para transformá-la em uma lista de 1 a 3 perguntas específicas, ajudando a fornecer uma resposta completa e detalhada.
      Análise e Contextualização: Identifique o objetivo central da pergunta, focando no conteúdo principal sem referências a comparações temporais ou anos específicos. Basicamente, quero que reflita: O que devo pedir para cada documento em ordem para obter resposta da pergunta do usuário.
      Decomposição em Sub-Perguntas: Quebre a pergunta original em sub-perguntas concisas e diretas.
      Para uma das subperguntas, mantenha o contexto central da pergunta, eliminando termos usuais de perguntas. Por exemplo, se a consulta for "Programação I saiu da grade?", uma das perguntas deve ser 'Programação I está na matriz curricular?"
      Regras:
      1. Ignore completamente comparações entre documentos ou dados temporais (ex.: PPC 2024 vs. PPC 2018). Evite totalmente termos como PPC 2024, PPC 2018 nas subconsultas geradas.
      2. Gere de 1 a 3 subconsultas.
      3. Caso a pergunta já seja específica o suficiente, não é necessário a transformação. Por exemplo: "Posso validar monitoria como ACC?" é um exemplo de perguntas que não fazem comparações entre os documentos. Já "O que mudou das ACCs de monitorias entre as versões do PPC" exige alterações.

      Por exemplo:
      Pergunta: "Quais disciplinas mudaram para o novo PPC?"

      Transformação: ["Qual é a Matriz curricular?", "Liste todos os componentes curriculares do curso", "Grade Curricular", "Componentes curriculares que não têm equivalência para a matriz anterior"]

      Revise a resposta gerada. Veja se atenda as regras específicadas.

      Pergunta: {query}

      Retorne apenas a lista python de perguntas transformadas, separadas por vírgulas, considerando as perguntas necessárias para cada documento que ajudem a responder completamente a questão original.
    """


    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])
    chain = prompt_template | query_expansion_llm | CommaSeparatedListOutputParser()
    result = chain.invoke({})
    print(result)
    return {
        'queries': result,
    }

def process_query_for_ppcs(
    query: str,
    responses: Dict[str, List[str]],
    pipeline_type: str
) -> Tuple[Dict[str, List[str]], Dict[str, int], str, List[Dict]]:
    """
    Process a single query for both PPC versions.

    Returns:
        - Updated responses
        - Token usage per PPC
        - Last context processed
        - List of raw token usage dictionaries
    """
    token_usages = []
    ppc_usage = {'ppc_2018': 0, 'ppc_2024': 0}

    # Process for PPC 2018
    context_2018, response_2018, _, usage_2018 = (
        naive_pipeline(query, 'ppc2018') if pipeline_type == "naive"
        else advanced_pipeline(query, 'ppc2018')
    )
    responses['2018'].append(response_2018)
    ppc_usage['ppc_2018'] = usage_2018.get('total_tokens', 0)
    token_usages.append(usage_2018)

    time.sleep(5)  # Rate limiting

    # Process for PPC 2024
    context_2024, response_2024, _, usage_2024 = (
        naive_pipeline(query, 'ppc2024') if pipeline_type == "naive"
        else advanced_pipeline(query, 'ppc2024')
    )
    responses['2024'].append(response_2024)
    ppc_usage['ppc_2024'] = usage_2024.get('total_tokens', 0)
    token_usages.append(usage_2024)

    time.sleep(5)  # Rate limiting

    return responses, ppc_usage, context_2024, token_usages

def generate_final_comparative_response(
    original_query: str,
    responses: Dict[str, List[str]]
) -> Tuple[str, Dict]:
    """
    Generate final comparative response from individual PPC responses.

    Returns:
        - Final formatted response
        - Token usage dictionary
    """


    prompt = """
    Você é um assistente especializado em responder dpuvidas sobre os PPCs de Ciência da Computação da UFFS.
    Analise as respostas fornecidas e gere uma resposta final.

    PPC 2018:
    {ppc_2018}

    PPC 2024:
    {ppc_2024}

    Regras:
    1. Foque apenas na pergunta do usuário
    2. Use markdown para formatação
    3. Seja conciso e informativo
    4. Responda apenas o solicitado
    5. Omita informações faltantes

    Pergunta original: {query}
    """

    prompt_data = {
        "ppc_2018": " ".join(responses['2018']),
        "ppc_2024": " ".join(responses['2024']),
        "query": original_query
    }

    final_context = (
        "PPC 2018 Context:\n" + "\n".join(responses['2018']) + CHUNK_SPLITTER +
        "PPC 2024 Context:\n" + "\n".join(responses['2024'])
    )

    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])
    chain = prompt_template | final_response_llm
    result = chain.invoke(prompt_data)

    return result.content, final_context, result.response_metadata.get('token_usage', {})









In [ ]:
from typing import Dict, Tuple, Optional, Union
import time
import traceback

def unified_pipeline(
    query: str,
    pipeline_type: str = "naive",
    source_name: Optional[str] = None
) -> Tuple[str, str, str, Dict]:
    """
    Unified pipeline that handles both specific PPC queries and comparative analysis.

    Args:
        query: The user's question
        source_name: Optional specific PPC source to query
        pipeline_type: Either "naive" or "advanced"

    Returns:
        Tuple containing:
        - context: Last relevant context used
        - response: Final response content
        - expanded_query: Original expanded query or transformed queries for comparison
        - token_usage: Dictionary with token usage information
    """
    # If source_name is provided, use specific PPC pipeline
    if source_name:
        if pipeline_type == "naive":
            context, response, expanded_query, token_usage = naive_pipeline(query, source_name)
        elif pipeline_type == "advanced":
            context, response, expanded_query, token_usage = advanced_pipeline(query, source_name)
        else:
            context, response, expanded_query, token_usage = naive_ft_pipeline(query, source_name)

        return context, response, expanded_query, token_usage

    # If no source_name, do comparative analysis
    try:
        # Get comparative response using the comparative pipeline
        final_response, expanded_queries, last_context, token_usage = get_comparative_answer(query, pipeline_type)

        return last_context, final_response, expanded_queries, token_usage

    except Exception as e:
        error_msg = f"Error in comparative analysis: {str(e)}"
        traceback.print_exc()

        return "", error_msg, "", {"error": str(e)}

In [ ]:
context, answer, _, tokens_used = unified_pipeline('Ementa de computação gráfica', 'advanced', 'ppc2018')
answer

Query expandida:  COMPUTAÇÃO GRÁFICA | EMENTA
5
0
content='A ementa de Computação Gráfica (GEX107) é:\n\nConceitos básicos. Dispositivos gráficos. Sistemas de cores. Transformações geométricas. Primitivas gráficas. Visibilidade. Rendering (modelos de iluminação, shading, textura, antialiasing).\n\nEssa informação foi retirada do PPC fornecido.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 3932, 'total_tokens': 4040, 'completion_tokens': 108}, 'model': 'open-mixtral-8x22b', 'finish_reason': 'stop'} id='run-259fc449-c6ef-4f77-905b-2cbadbe2f41a-0' usage_metadata={'input_tokens': 3932, 'output_tokens': 108, 'total_tokens': 4040}


'A ementa de Computação Gráfica (GEX107) é:\n\nConceitos básicos. Dispositivos gráficos. Sistemas de cores. Transformações geométricas. Primitivas gráficas. Visibilidade. Rendering (modelos de iluminação, shading, textura, antialiasing).\n\nEssa informação foi retirada do PPC fornecido.'

In [ ]:
context, answer, expanded_queries, tokens_used = unified_pipeline('Quais CCRs possuem equivalência com o PPC 2018?', 'advanced')
answer

['["Quais CCRs possuem equivalência?"', '"Liste todos os CCRs do curso"', '"CCR que possuem equivalência com a matriz anterior"]']
['Quais CCRs possuem equivalência com o PPC 2018?', '["Quais CCRs possuem equivalência?"', '"Liste todos os CCRs do curso"', '"CCR que possuem equivalência com a matriz anterior"]']
Query expandida:  CURRÍCULO COMPLEMENTAR | DISCIPLINAS COM EQUIVALÊNCIA NO PPC 2018
5
2
content='Os CCRs que possuem equivalência com o PPC 2018, conforme a tabela do Art. 1º, são:\n\n- GEN254 - Grafos (4 créditos)\n- GEX615 - Engenharia de software I (4 créditos)\n- GCH290 - Iniciação à prática científica (4 créditos)\n- GCS239 - Direitos e cidadania (4 créditos)\n- GCH292 - História da fronteira Sul (4 créditos)\n- GEX616 - Engenharia de software II (4 créditos)\n- GCS580 - Planejamento e gestão de projetos (4 créditos)\n- GEX618 - Inteligência artificial (4 créditos)\n- GCS238 - Meio ambiente, economia e sociedade (4 créditos)\n- GEX620 - Arquitetura de computadores (4 crédit

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py", line 72, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/connection_pool.py", line 216, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/connection.py", line 101, in handle_request
    return self._connection.handle_request(request)
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/http11.py", line 143, in handle_request
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/http11.py", line 113, in handle_request
    ) = self._receive_

'Error in comparative analysis: The read operation timed out'

In [ ]:
context, answer, expanded_queries, tokens_used

('',
 'Error in comparative analysis: The read operation timed out',
 '',
 {'error': 'The read operation timed out'})

## Perguntas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mkdir -p data
!cp /content/drive/My\ Drive/tcc/validacao/GroundTruthPPC2018_Diretrizes_Refinadas.csv data/
!cp /content/drive/My\ Drive/tcc/validacao/GroundTruthPPC2024_Diretrizes_Refinadas.csv data/
!cp /content/drive/My\ Drive/tcc/validacao/GroundTruthPPCAmbos_Diretrizes_Refinadas.csv data/


In [ ]:
import pandas as pd

df1 = pd.read_csv('data/GroundTruthPPC2018_Diretrizes_Refinadas.csv')
df2 = pd.read_csv('data/GroundTruthPPC2024_Diretrizes_Refinadas.csv')
df3 = pd.read_csv('data/GroundTruthPPCAmbos_Diretrizes_Refinadas.csv')

df1['source'] = 'ppc2018'
df2['source'] = 'ppc2024'
df3['source'] = 'Ambos'

questions_df = pd.concat([df1, df2, df3], ignore_index=True)


In [ ]:
questions_df

,Pergunta,Resposta,Diretriz_de_Avaliacao,source
0,Qual é a ementa de Informática Básica?,Fundamentos de informática. Conhecimentos de s...,A resposta ideal deve conter os seguintes pont...,ppc2018
1,Qual é a ementa de Matemática C?,Grandezas proporcionais. Noções de geometria. ...,A resposta ideal deve conter os seguintes pont...,ppc2018
2,Qual é a ementa de Introdução à Filosofia?,A natureza e especificidade do discurso filosó...,A resposta ideal deve conter os seguintes pont...,ppc2018
3,Qual é a ementa de Algoritmos e Programação?,Conceito e construção de algoritmos. Tipos bás...,A resposta ideal deve conter os seguintes pont...,ppc2018
4,Qual é a ementa de Circuitos Digitais?,Fundamentos de Eletrônica. Famílias Lógicas. S...,A resposta ideal deve conter os seguintes pont...,ppc2018
...,...,...,...,...
115,Ministrar cursos conta como extensão?,"No PPC 2024, ministrar cursos de extensão cont...",A resposta ideal deve conter os seguintes pont...,Ambos
116,Engenharia de software II saiu da grade?,A disciplina Engenharia de Software II foi rem...,A resposta ideal deve conter os seguintes pont...,Ambos
117,Pesquisa e ordenação de dados foi removida? Ou...,A disciplina Pesquisa e Ordenação de Dados não...,A resposta ideal deve conter os seguintes pont...,Ambos
118,Participação em projetos de iniciação científi...,"No PPC 2024, o limite de horas para iniciação ...",A resposta ideal deve conter os seguintes pont...,Ambos


In [ ]:
import pandas as pd
import csv
from time import sleep

def process_csv_and_generate_answers(question_df, inference_config, rag_solution='advanced', retries=1):
    output_csv_path = rag_solution + '.csv'

    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ["Documento", "Pergunta", "RespostaIdeal", "Diretriz_de_Avaliacao", "RespostaGerada", "PromptTokens", "TotalTokens", "CompletionTokens", "Contexto", "ConsultaExpandida", "ConfiguracaoDeInferencia", "Metodo"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()

        for index, row in question_df.iterrows():
            question = row['Pergunta']
            goal_response = row['Resposta'] if not pd.isna(row['Resposta']) else ''
            evaluation_guideline = row['Diretriz_de_Avaliacao'] if not pd.isna(row['Diretriz_de_Avaliacao']) else ''
            source_name = row['source']

            if source_name == 'Ambos':
                source_name = None

            for attempt in range(retries):
                try:
                    context, response, expanded_query, token_usage = unified_pipeline(question, rag_solution, source_name)

                    prompt_tokens = token_usage.get('prompt_tokens')
                    total_tokens = token_usage.get('total_tokens')
                    completion_tokens = token_usage.get('completion_tokens')

                    writer.writerow({
                        "Documento": row['source'],
                        "Pergunta": question,
                        "RespostaIdeal": goal_response,
                        "Diretriz_de_Avaliacao": evaluation_guideline,
                        "RespostaGerada": response,
                        "PromptTokens": prompt_tokens,
                        "TotalTokens": total_tokens,
                        "CompletionTokens": completion_tokens,
                        "ConsultaExpandida": expanded_query,
                        "Contexto": context,
                        "ConfiguracaoDeInferencia": inference_config,
                        "Metodo": rag_solution
                    })

                    print(f"Pergunta: {question}")
                    print(f"Resposta: {response}")
                    sleep(20)

                    break  # Break out of the retry loop if successful
                except Exception as e:
                    print(f"Erro ao processar a pergunta: {question}. Tentativa {attempt + 1} de {retries}. Erro: {e}")
                    if attempt == retries - 1:
                        sleep(40)
                        print(f"Falha ao processar a pergunta: {question} após {retries} tentativas.")

    print(f"Novo CSV salvo com sucesso em {output_csv_path}.")


In [ ]:
# NAIVE_INFERENCE_CONFIG = 'NAIVE: open-mixtral-8x22b Contexto: 5. PINECONE normal'

# ADVANCED_INFERENCE_CONFIG = 'ADVANCED: open-mixtral-8x22b. Transformação: Gemma2-9b-it, Contexto: 5, 5. PINECONE com metadados'

In [ ]:
NAIVE_INFERENCE_CONFIG = 'NAIVE: Expansão: llama-3.1-70b, RAG: open-mixtral-8x22b, Final:mistral-large-latest Contexto: 5. PINECONE normal, embedding zero shot'
NAIVE_FT_INFERENCE_CONFIG = 'NAIVE: Expansão: llama-3.1-70b, RAG: open-mixtral-8x22b, Final:mistral-large-latest Contexto: 5. PINECONE normal, embedding com fine tuning'
ADVANCED_INFERENCE_CONFIG = 'ADVANCED:  Expansão: llama-3.1-70b, RAG: open-mixtral-8x22b  Transformação: mixtral-8x7b , Final:mistral-large-latest, Contexto: 5, 5. PINECONE com metadados. Embeddings: winderfeld/cc-uffs-ppc-ft-test-multiqa'

In [ ]:
process_csv_and_generate_answers(questions_df, inference_config = NAIVE_FT_INFERENCE_CONFIG , rag_solution='naiveft')

{'_embedding': HuggingFaceEmbeddings(model_name='winderfeld/cc-uffs-ppc-ft-test-multiqa', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), '_text_key': 'text', '_namespace': None, 'distance_strategy': <DistanceStrategy.COSINE: 'COSINE'>, '_index': <pinecone.data.index.Index object at 0x7e276857bd90>}
content='A ementa de Informática Básica é: "Fundamentos de informática. Conhecimentos de sistemas operacionais. Utilização da rede mundial de computadores. Ambientes virtuais de aprendizagem. Conhecimentos de softwares de produtividade para criação de projetos educativos e/ou técnicos e/ou multimidiáticos."' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 4838, 'total_tokens': 4935, 'completion_tokens': 97}, 'model': 'open-mixtral-8x22b', 'finish_reason': 'stop'} id='run-91cbda04-e8dc-4dac-aefe-937d36d756a3-0' usage_metadata={'input_tokens': 4838, 'output_tokens': 97, 'total_tokens': 4935}
Pergunta: Qual é a ementa de

In [ ]:
# process_csv_and_generate_answers(questions_df, inference_config = NAIVE_INFERENCE_CONFIG , rag_solution='naive')

In [ ]:
process_csv_and_generate_answers(questions_df, ADVANCED_INFERENCE_CONFIG, rag_solution='advanced')

Query expandida:  INFORMÁTICA BÁSICA | EMENTA
5
0
content='A ementa de Informática Básica é: "Fundamentos de informática. Conhecimentos de sistemas operacionais. Utilização da rede mundial de computadores. Ambientes virtuais de aprendizagem. Conhecimentos de softwares de produtividade para criação de projetos educativos e/ou técnicos e/ou multimidiáticos."' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 3789, 'total_tokens': 3886, 'completion_tokens': 97}, 'model': 'open-mixtral-8x22b', 'finish_reason': 'stop'} id='run-73f5e72d-0814-45b2-bfed-6857a04d0c4a-0' usage_metadata={'input_tokens': 3789, 'output_tokens': 97, 'total_tokens': 3886}
Pergunta: Qual é a ementa de Informática Básica?
Resposta: A ementa de Informática Básica é: "Fundamentos de informática. Conhecimentos de sistemas operacionais. Utilização da rede mundial de computadores. Ambientes virtuais de aprendizagem. Conhecimentos de softwares de produtividade para criação de projetos educativos e/ou t

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py", line 72, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.10/dist-packages/httpx/_transports/default.py", line 236, in handle_request
    resp = self._pool.handle_request(req)
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/connection_pool.py", line 216, in handle_request
    raise exc from None
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/connection.py", line 101, in handle_request
    return self._connection.handle_request(request)
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/http11.py", line 143, in handle_request
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/httpcore/_sync/http11.py", line 113, in handle_request
    ) = self._receive_

Pergunta: Quais CCRs possuem equivalência com o PPC 2018?
Resposta: Error in comparative analysis: The read operation timed out
['["Componentes curriculares da matriz curricular atual"', '"Componentes curriculares da matriz curricular anterior"]']
['Componentes curriculares que não têm equivalência para a matriz do PPC antigo?', '["Componentes curriculares da matriz curricular atual"', '"Componentes curriculares da matriz curricular anterior"]']
Query expandida:  COMPONENTES CURRICULARES SEM EQUIVALÊNCIA PARA A MATRIZ DO PLANO PEDAGÓGICO DO CURSO ANTIGO
5
4
content='A resposta para a pergunta "Componentes curriculares que não têm equivalência para a matriz do PPC antigo?" não foi encontrada nos trechos fornecidos do PPC.' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 1758, 'total_tokens': 1805, 'completion_tokens': 47}, 'model': 'open-mixtral-8x22b', 'finish_reason': 'stop'} id='run-98ea5079-205d-4ec4-9998-20c33f775c08-0' usage_metadata={'input_tokens': 1758,

In [ ]:
import shutil
from datetime import datetime
import pytz

def save_csv_to_drive(csv_file_path, drive_folder_path):
    # Define the timezone for BRT
    brt_timezone = pytz.timezone('America/Sao_Paulo')

    file_name = csv_file_path.split('.csv')[0]
    file_name = file_name.upper()

    # Generate a timestamp in BRT timezone
    timestamp = datetime.now(brt_timezone).strftime('%d_%m_%Y_%H-%M-%S')

    # Define the path in Google Drive with timestamp
    drive_path = f'{drive_folder_path}/{file_name}_{timestamp}.csv'

    # Copy the CSV file to Google Drive
    shutil.copy(csv_file_path, drive_path)
    print(f'File saved to: {drive_path}')



In [ ]:
'a'.upper()

'A'

In [ ]:
drive_folder_path = '/content/drive/MyDrive/tcc/resultados'

In [ ]:
save_csv_to_drive('naive.csv', drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/resultados/NAIVE_03_11_2024_10-55-05.csv


In [ ]:
save_csv_to_drive('naiveft.csv', drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/resultados/NAIVEFT_07_11_2024_22-39-13.csv


In [ ]:
save_csv_to_drive('advanced.csv', drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/resultados/ADVANCED_12_11_2024_22-21-09.csv
